In [56]:
! pip install pyspark

In [57]:
import requests
import io
import pandas as pd
from pyspark.sql import SparkSession

In [58]:
url = 'http://compras.dados.gov.br/servicos/v1/servicos.csv?descricao=educação'

In [59]:
data = requests.get(url).content
data

b'C\xc3\xb3digo,Descri\xc3\xa7\xc3\xa3o,Unidade medida,CPC,Se\xc3\xa7\xc3\xa3o,Divis\xc3\xa3o,Grupo,Classe,Subclasse\n14311,"Orienta\xc3\xa7\xc3\xa3o / Educa\xc3\xa7\xc3\xa3o - Atividade F\xc3\xadsica                                                                                ",UNHOMEM/H M\xc3\x8aS,9659,"9: SERVI\xc3\x87OS DA COMUNIDADE, PESSOAIS E SOCIAIS E OUTROS SERVI\xc3\x87OS                                                            ","96: SERVI\xc3\x87OS RECREACIONAIS, CULTURAIS E ESPORTIVOS                                                                          ","965: SERVI\xc3\x87OS RELACIONADOS COM ESPORTES E SERVI\xc3\x87OS RECREACIONAIS DO ESPORTE                                                  ","9659: OUTROS ESPORTES E SERVI\xc3\x87OS DE ESPORTES RECREATIVOS                                                                      ",\n15946,"Servi\xc3\xa7o Penitenci\xc3\xa1rio - Reeduca\xc3\xa7\xc3\xa3o e Reabilita\xc3\xa7\xc3\xa3o de Apenado                             

In [60]:
parse_data = io.StringIO(data.decode('utf-8'))
print (parse_data)

In [61]:
df = pd.read_csv(parse_data)
df.head()

,Código,Descrição,Unidade medida,CPC,Seção,Divisão,Grupo,Classe,Subclasse
0,14311,Orientação / Educação - Atividade Física ...,UNHOMEM/H MÊS,9659,"9: SERVIÇOS DA COMUNIDADE, PESSOAIS E SOCIAIS ...","96: SERVIÇOS RECREACIONAIS, CULTURAIS E ESPORT...",965: SERVIÇOS RELACIONADOS COM ESPORTES E SERV...,9659: OUTROS ESPORTES E SERVIÇOS DE ESPORTES R...,NaN
1,15946,Serviço Penitenciário - Reeducação e Reabilita...,UN,979,"9: SERVIÇOS DA COMUNIDADE, PESSOAIS E SOCIAIS ...",97: OUTROS SERVIÇOS ...,979: OUTROS SERVIÇOS DIVERSOS/MISCELÂNEA ...,NaN,NaN
2,18481,Consultoria e Assessoria - Educação ...,UN,8319,"8: SERVIÇOS DE ARQUITETURA, DESENHOS TÉCNICOS,...","83: OUTROS SERVIÇOS DE NEGÓCIOS, TÉCNICOS E PR...",831: SERVIÇOS DE CONSULTORIA E DE GERÊNCIA/GES...,"8319: OUTROS SERVIÇOS DE GERÊNCIA/GESTÃO, EXCE...",NaN
3,19321,Curso / Treinamento Educação - Distância ...,UN,929,"9: SERVIÇOS DA COMUNIDADE, PESSOAIS E SOCIAIS ...",92: SERVIÇOS DA INSTRUÇÃO/ EDUCAÇÃO/ ENSINO ...,929: OUTROS SERVIÇOS DE EDUCAÇÃO E TREINAMENTO...,NaN,NaN


In [62]:
spark = SparkSession.builder.appName("Aula").getOrCreate()

In [63]:
df = spark.createDataFrame(df)

In [64]:
df.printSchema()

root
 |-- Código: long (nullable = true)
 |-- Descrição: string (nullable = true)
 |-- Unidade medida: string (nullable = true)
 |-- CPC: long (nullable = true)
 |-- Seção: string (nullable = true)
 |-- Divisão: string (nullable = true)
 |-- Grupo: string (nullable = true)
 |-- Classe: string (nullable = true)
 |-- Subclasse: double (nullable = true)



In [67]:
df = (df.withColumnRenamed("Código", "Codigo")
.withColumnRenamed("Descrição","Descricao")
.withColumnRenamed("Unidade medida", "Unidade_Medida")
.withColumnRenamed("Seção", "secao")
.withColumnRenamed("Divisão", "Divisao")
)

In [71]:
df = df.select([x.lower() for x in df.columns])
df.printSchema()

root
 |-- codigo: long (nullable = true)
 |-- descricao: string (nullable = true)
 |-- unidade_medida: string (nullable = true)
 |-- cpc: long (nullable = true)
 |-- secao: string (nullable = true)
 |-- divisao: string (nullable = true)
 |-- grupo: string (nullable = true)
 |-- classe: string (nullable = true)
 |-- subclasse: double (nullable = true)



In [72]:
from google.colab import drive

In [78]:
import os
os.makedirs('/content/drive/MyDrive/xpe/data', exist_ok=True)
drive.mount("/content/drive/MyDrive/xpe/data")

Mounted at /content/drive/MyDrive/xpe/data


In [79]:
df.write.mode("overwrite").csv("/content/drive/MyDrive/xpe/data/educacao.csv")